# Visualize baseline methods
**Description:** This notebook presents the baseline methods available in `snazzy_analysis` and the resulting DFF signals.  

## Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import matplotlib.pyplot as plt

from snazzy_analysis import Dataset, Trace

## Load Data

Specify the location of the data by editing `dataset_name`. Then specify the embryo.

In [ ]:
dataset_name = "20240611"  # HERE

dataset_path = Path.cwd().parent.joinpath("data", dataset_name)
dataset = Dataset(dataset_path)

embryos = list(dataset.embryos)
emb = embryos[0]  # HERE
trace = emb.trace

print(f"\n\nLoading {dataset_name} {emb.name}")

## Baseline Methods

### Local minima baseline

The baseline value at time `t` is the average of the `n` lowest values, selected from a window of length `ws`, centered at point `t`.

### Most frequent bin average baseline

The baseline value at time `t` is the average of the values that fall inside the most frequent bin.
The first step is to calculate the histogram from a window of length `ws`.
The values of the bin with the highest number of elements are averaged and used as the baseline.
This assumes that the peaks are sparse in comparison to the baseline, so that when you pick the most frequent bin it will fall under the baseline range.


In [ ]:
emb_index = 0  # Change emb_index to use another embryo

emb = embryos[emb_index]
trace = emb.trace

ws = 161
num_minima_points = 21

fig, (ax0, ax1) = plt.subplots(2, figsize=(10, 10))

fig.suptitle(emb.name)

ratiom_signal = trace.compute_ratiom_gcamp()
local_minima_baseline = Trace.average_n_lowest_window(
    ratiom_signal, ws, num_minima_points
)
most_freq_baseline = trace.compute_baseline(ratiom_signal, ws)

ax0.plot(ratiom_signal, label="Ratiometric signal", color="green")
ax0.plot(local_minima_baseline, label=f"Average lower {num_minima_points}")
ax0.plot(most_freq_baseline, label="Most frequent values")
ax0.axvline(trace.trim_idx, color="r")
ax0.legend()

dff_local_minima = (ratiom_signal - local_minima_baseline) / local_minima_baseline
dff_most_freq = (ratiom_signal - most_freq_baseline) / most_freq_baseline

ax1.plot(dff_local_minima, label=f"DFF (Average lower {num_minima_points})")
ax1.plot(dff_most_freq, label="DFF (Most frequent values)")
ax1.axvline(trace.trim_idx, color="r")
ax1.legend()

plt.show()

As can be seen in the plots above, the local minima method creates a smoother baseline.
This directly impacts on the resulting DFF signal (dff = (ratiometric_signal - baseline) / baseline).
The most frequent bin baseline method makes much stronger assumptions about the ratiometric signal distribution, which means that it tends to perform worse when the signal deviates from these assumptions.
The local minima method is more general, and tends to produce a baseline with values that follow the ratiometric signal more closely.